In [1]:
#Import base packages
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset, MonthEnd
import scipy.linalg as la
from scipy.stats import norm
import time

#Set seeds for different algorithms
import random
random.seed(200)
np.random.seed(200)

#Setup directory and pickling
import os
import pickle

#Help ignore the high verbosity in optuna
import warnings
warnings.filterwarnings("ignore")

#Import sklearn machine learning packages
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import RobustScaler

#Import LGBM
!pip install lightgbm
import lightgbm
from lightgbm import LGBMRegressor

#Import XGBoost
!pip install xgboost
import xgboost
from xgboost import XGBRegressor

#Import Optuna tuning hyperparameter
!pip install optuna
import optuna
from optuna.visualization import plot_optimization_history, plot_slice
optuna.logging.set_verbosity(optuna.logging.WARNING)
from optuna.samplers import TPESampler

#Keras imports for NN
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from tensorflow.python.keras import regularizers
import keras
from keras import layers
from keras import models
from keras import utils
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from keras import datasets
from keras.callbacks import LearningRateScheduler
from keras.callbacks import History
from keras import losses
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, CSVLogger, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

!pip install numpy
!pip install tensorflow

from tensorflow.keras.layers import Dense, LSTM, Conv2D, Flatten, Dropout, Activation, Reshape, TimeDistributed, Conv1D, RepeatVector
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import History

#Setup an initial keras random seed - will be superceded by other random seeds in ensembling
tf.keras.utils.set_random_seed(0)

#The below objectives set the optimization criteria by model
#'n_components':[i for i in range(1, 31, 1)]
#!pip install optkeras
#import optkeras
#optkeras.optkeras.get_trial_default = lambda: optuna.trial.FrozenTrial(
#            None, None, None, None, None, None, None, None, None, None, None)
#from optkeras.optkeras import OptKeras
#import tensorflow.keras.backend as K
#from tensorflow.keras.callbacks import Callback, CSVLogger, ModelCheckpoint
#from optkeras.optkeras import OptKeras
#sorted(sklearn.metrics.SCORERS.keys())


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Upload Stock & Bond data
sb_df = pd.read_csv("C:/Users/andrew_lazzeri/Desktop/MultiAsset Covariance Matrix/Stock_Bond_Rets.csv")
sb_df['Date'] = pd.to_datetime(sb_df['Unnamed: 0'])
sb_df = sb_df.drop(['Unnamed: 0', 'MLPs', 'REITs', 'TIPS'], axis=1)
sb_df = sb_df.set_index(['Date'])
sb_synched_df = pd.DataFrame(sb_df.iloc[2::, :].astype(float)/100, columns = sb_df.columns)
sb_synched_df = sb_synched_df.dropna()

#Upload Commodity Data
commodity_df = pd.read_csv("C:/Users/andrew_lazzeri/Desktop/MultiAsset Covariance Matrix/CommodityIndex.csv")
commodity_df['Date'] = pd.to_datetime(commodity_df['Date'].astype(str).str[0:8])
commodity_df = commodity_df.set_index('Date')

#Create a combined dataframe
combined_df = pd.DataFrame(index = sb_synched_df.index)
combined_df = combined_df.join(commodity_df)
combined_df = combined_df.pct_change(periods = 1)
combined_df = combined_df.join(sb_synched_df)

In [3]:
## Set Parameters ##
cov_window = 63 #Lookback window for the covariance matrix
days_fwd = 63 #Forward window for realized covariance matrix target
annualization_adj = 252 #Adjustment to annualize the covariance matrix (i.e. 252 for trading days)
initial_train_window = 2*252 #Required observations prior to first training
retrain_intervals = 252 #Amount of time between training intervals
CV_budget = 3 #Cross validations hyper parameter tuning for each algorithm

In [4]:
#Generating a series of covariance matrix
def create_tr_covar(combined_df, look_back, annualization_adj):
    covar_mat = combined_df.rolling(window = look_back).cov()
    covar_mat = covar_mat.dropna() * annualization_adj
    covar_mat.index.rename(['Date', 'Commodity'], inplace=True)
    return covar_mat

def create_exp_tr_covar(combined_df, look_back, annualization_adj):
    exp_covar_mat = combined_df.ewm(span=look_back).cov()
    exp_covar_mat = exp_covar_mat.dropna() * annualization_adj
    exp_covar_mat.index.rename(['Date', 'Commodity'], inplace=True)
    return exp_covar_mat

def Y_variable(covar_mat, fwd_period):
    y_covar_mat = covar_mat.copy()
    y_covar_mat = y_covar_mat.reset_index()
    y_covar_mat['Date'] = y_covar_mat['Date'] - DateOffset(days = fwd_period)
    y_covar_mat = y_covar_mat.set_index(['Date', 'Commodity'])
    return y_covar_mat

def data_transform(original_mat):
    Lambda, Q = np.linalg.eig(original_mat)
    S = np.dot(Q,np.dot(np.diag(np.sqrt(Lambda)),Q.T))
    C = la.cholesky(S)
    return C

def to_rcov(transformed_mat):
    S = np.dot(transformed_mat.T, transformed_mat)
    Lambda, Q = np.linalg.eig(S)
    rcov = np.dot(Q,np.dot(np.diag(np.square(Lambda)),Q.T))
    return rcov

def generate_matrix(covar_mat):
    dates = sorted(list(set(covar_mat.index.get_level_values('Date'))))
    transformed_df = pd.DataFrame()
    for date in dates:
        iter_df = covar_mat[covar_mat.index.get_level_values('Date')==date]
        try:
            cholesky_mat = data_transform(iter_df)
            cholesky_df = pd.DataFrame(cholesky_mat, index = iter_df.index, columns = iter_df.columns)
            transformed_df = pd.concat([transformed_df, cholesky_df])
        except:
            continue
            
    return transformed_df 

def ML_feature_generator(cov_df:pd.DataFrame()) -> pd.DataFrame():
    melt_C_df = pd.melt(cov_df.reset_index(), id_vars = ['Date', 'Commodity'])
    melt_C_df = melt_C_df[melt_C_df['value']!=0]
    melt_C_df['FinalIdx'] = melt_C_df['Commodity'].str.split("_", expand = True)[0] + '_' + melt_C_df['variable'].str.split("_", expand = True)[0] 
    melt_C_df = melt_C_df[['Date','value', 'FinalIdx']]
    melt_C_df = melt_C_df.sort_values(['Date', 'FinalIdx'], ascending = True)
    melt_C_df = melt_C_df.set_index(['Date', 'FinalIdx'])
    return melt_C_df

def add_lags(cov_df, period_len, lags):
    iter_df = cov_df.copy()
    dates = sorted(list(set(iter_df.index.get_level_values('Date'))))
    lag_df = pd.DataFrame(index = dates)
    iter_pivot = iter_df.reset_index().pivot(index = 'Date', columns = 'FinalIdx', values = 'value')
    
    for i in range(0, lags*period_len+1, period_len):
        lag_add = iter_pivot.shift(i).copy()
        lag_add = lag_add.add_suffix('Lag_'+str(i))
        lag_df = lag_df.join(lag_add)
        
    return lag_df

def backfill_covar(covar_df, resample_freq):
    covar_copy = covar_df.copy()
    min_date = min(covar_df.index.get_level_values('Date'))
    max_date = max(covar_df.index.get_level_values('Date'))
    dates = pd.date_range(min_date, max_date, freq = resample_freq)
    resampled_df = pd.DataFrame()
    
    for i in range(len(dates)):
        if (covar_df[covar_df.index.get_level_values('Date') == dates[i]].shape[0] != 0):
            iter_covar_df = covar_copy[covar_copy.index.get_level_values('Date') == dates[i]]
            resampled_df = pd.concat([resampled_df, iter_covar_df])
        elif (covar_df[covar_df.index.get_level_values('Date') == dates[i]].shape[0] == 0):
            last_time_stamp = max(resampled_df.index.get_level_values('Date'))
            iter_covar_df = resampled_df[resampled_df.index.get_level_values('Date') == last_time_stamp]
            iter_covar_df = iter_covar_df.reset_index()
            iter_covar_df['Date'] = pd.to_datetime(dates[i])
            iter_covar_df = iter_covar_df.set_index(['Date', 'Commodity'])
            resampled_df = pd.concat([resampled_df, iter_covar_df])
   
    return resampled_df

X_df = generate_matrix(create_tr_covar(combined_df, cov_window, annualization_adj))
y_df = generate_matrix(Y_variable(create_tr_covar(combined_df, cov_window, annualization_adj), days_fwd))
y_df = backfill_covar(y_df, 'D')
C_df_X = ML_feature_generator(X_df)
C_df_y = ML_feature_generator(y_df)
C_df_X_lag = add_lags(C_df_X, 5, 52)
df_y = C_df_y.reset_index().pivot(index = 'Date', columns = 'FinalIdx', values = 'value')
df_y = df_y.add_prefix('Y_')
combined_df = C_df_X_lag.join(df_y)
combined_df = combined_df.dropna()

In [5]:
def generate_df_splits(combined_df, date):
    #Split the data by time
    train_df = combined_df[combined_df.index < date]
    predict_df = combined_df[combined_df.index == date]
    
    #Split the data by features and targets
    train_x = train_df[[col for col in train_df.columns if 'Y_' not in col]]
    train_y = train_df[[col for col in train_df.columns if 'Y_' in col]]
    predict_x = predict_df[[col for col in predict_df.columns if 'Y_' not in col]]
    predict_y = predict_df[[col for col in predict_df.columns if 'Y_' in col]]
    
    return train_x, train_y, predict_x, predict_y

#This simply creates an object to avoid an error downstream and is irrelevant otherwise
train_x, train_y, predict_x, predict_y = generate_df_splits(combined_df, '2003-12-31')

def apply_robust_scaler(train_x, predict_x):
    transformer = RobustScaler(quantile_range=(1.0, 99.0)).fit(train_x)
    train_x = pd.DataFrame(transformer.transform(train_x), index = train_x.index, columns = train_x.columns)
    predict_x = pd.DataFrame(transformer.transform(predict_x), index = predict_x.index, columns = predict_x.columns)
    return train_x, predict_x

def ts_cross_validation(train_x, train_y, model, ts_folds = 3):
    combined_df = train_x.join(train_y)
    col_len = len(combined_df.index)
    rough_splits = col_len // (ts_folds + 1)
    errors = []
    idx = [i for i in range(0, combined_df.shape[0], rough_splits)]
    
    for i in range(2, len(idx)):
        iter_train = combined_df.iloc[idx[i-2]:idx[i-1], :] 
        iter_test = combined_df.iloc[idx[i-1]:idx[i], :]
        iter_train_x = iter_train[train_x.columns]
        iter_test_x = iter_test[train_x.columns]
        iter_train_x, iter_test_x = apply_robust_scaler(iter_train_x, iter_test_x)
        
        model.fit(iter_train_x, iter_train[train_y.columns])
        MSE = mean_squared_error(iter_test[train_y.columns], np.nan_to_num(model.predict(iter_test_x)))
        errors.append(MSE)
    return np.mean(errors)

def objective_XGB(trial, train_x = train_x, train_y = train_y):
    
    XGB_params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [x for x in range(50, 1000, 50)]),
        'learning_rate': trial.suggest_categorical('learning_rate', list(np.logspace(-2, .1, 20))),
        'max_depth': trial.suggest_categorical('max_depth', [x for x in range(1, 50)]),
        'subsample': trial.suggest_categorical('subsample', [x/10 for x in range(1, 10)]),
        'reg_lambda': trial.suggest_categorical('reg_lambda', list(np.logspace(0, 2, 20))),
        'reg_alpha': trial.suggest_categorical('reg_alpha', list(np.logspace(0, 2, 20))),
        'gamma': trial.suggest_categorical('gamma', list(np.logspace(-2, -.25, 20))),
        'verbosity': trial.suggest_categorical('verbosity', [0])
    }

    model = XGBRegressor(**XGB_params)  
    MSE = ts_cross_validation(train_x, train_y, model)
    return MSE

def objective_NN3(trial, train_x = train_x, train_y = train_y):
    K.clear_session() 
            
    #Setup the model hyperparameter search
    activation_fx = trial.suggest_categorical("activation", ["relu", "selu"])
    layer_units = trial.suggest_categorical('units', [64*4, 64*3, 64*2, 64*1])
    lr = trial.suggest_categorical('learning_rate', [.001, .01, .1])
    reg = trial.suggest_categorical('reg', [10**-3, 10**-5])
    droprate = trial.suggest_categorical('dropout', [.20, .30, .4])
    epochs = trial.suggest_categorical('epochs', [100])
    batch_n = trial.suggest_categorical('batch_size', [64, 64*2])
    es = EarlyStopping(monitor='val_loss', mode='min', patience = 10, restore_best_weights=True)
        
    #Setup the model structure
    model = Sequential()
    model.add(Dense(units = layer_units, activation = activation_fx,  kernel_initializer='he_uniform',
                    kernel_regularizer= regularizers.L1L2(l1=reg, l2=reg)))
    model.add(Dropout(droprate))
    model.add(BatchNormalization())
    model.add(Dense(units = layer_units/2, activation = activation_fx,  kernel_initializer='he_uniform', 
                    kernel_regularizer= regularizers.L1L2(l1=reg, l2=reg)))
    model.add(Dropout(droprate))
    model.add(BatchNormalization())
    model.add(Dense(units = layer_units/4, activation = activation_fx,  kernel_initializer='he_uniform',
                    kernel_regularizer= regularizers.L1L2(l1=reg, l2=reg)))
    model.add(Dropout(droprate))
    model.add(BatchNormalization())
    model.add(Dense(train_y.shape[1], kernel_initializer='normal',activation='linear'))
    model.compile(optimizer = Adam(learning_rate = lr), loss='mean_squared_error', metrics=['mean_squared_error'])
    model.fit(train_x, train_y, validation_split = 0.2, shuffle = True, batch_size = batch_n, 
              epochs = epochs, verbose = 0, callbacks=[es])  
    
    MSE = ts_cross_validation(train_x, train_y, model)
    return MSE


def optimization_instance(objective_fx, trial_runs: int):
    sampler = TPESampler(seed=200)  # Make the sampler behave in a deterministic way.
    study = optuna.create_study(sampler=sampler, direction='minimize') 
    study.optimize(objective_fx, n_trials=trial_runs)
    if len(study.trials) % 50 == 0:
        print(str(objective_fx), " has completed ", len(study.trials)/trial_runs, "of tuning trials.")
 
    return study

def evaluation(study, model, train_x, train_y, predict_x, predict_y):
    predict_model = model(**study.best_params)
    train_x, predict_x = apply_robust_scaler(train_x, predict_x)
    predict_model.fit(train_x, train_y)
    #Generate a test score and prediction
    y_pred = predict_model.predict(predict_x)
    MSE = mean_squared_error(predict_y, np.nan_to_num(y_pred))
    return y_pred, MSE, predict_y, predict_model

def save_predictions(prediction_df, model_name, date, working_path):
    #Save the prediction_df
    prediction_df.to_csv(working_path + "Prediction_dfs/" + model_name + '_' + date + '.csv')
    
def save_ML_model(model, model_name, date, working_path):
    filename = working_path + "Models/" + model_name + '_' + date + '.sav'
    pickle.dump(model, open(filename, 'wb'))      
    
def save_NN_model(model, model_name, date, working_path):
    filename = working_path + "Models/" + model_name + '_' + date + '.h'
    model.save(filename)

def backtest_XGB(combined_df, min_train_window, re_train_periods, runs):
    dates = sorted(set(list(combined_df.index)))
    model = XGBRegressor
    prediction_df = pd.DataFrame(columns = [col for col in combined_df.columns if 'Y_' in col])
       
    for i in range(min_train_window,len(dates)):
        train_x, train_y, predict_x, predict_y = generate_df_splits(combined_df, dates[i])
               
        if i % re_train_periods == 0:
            #Train the XGBoost Model
            start_time = time.time()
            study = optimization_instance(objective_XGB, runs)
            train_x, predict_x = apply_robust_scaler(train_x, predict_x)
            y_pred, MSE, predict_y, predict_model = evaluation(study, model, train_x, train_y, predict_x, predict_y)
            prediction_df = pd.concat([prediction_df, pd.DataFrame(y_pred, columns = train_y.columns, index = predict_x.index)])
            working_path = "C:/Users/andrew_lazzeri/Desktop/MultiAsset Covariance Matrix/"
            save_predictions(prediction_df, 'XGB', str(dates[i])[0:10], working_path)
            save_ML_model(predict_model, 'XGB', str(dates[i])[0:10], working_path)
            end_time = time.time()
            print("One training instance takes: ", (end_time - start_time)/60.0)
            print(str(np.round(i/len(dates) ,2)), " of the run is complete.")
                      
        else:
            train_x, predict_x = apply_robust_scaler(train_x, predict_x)
            y_pred = predict_model.predict(predict_x)
            prediction_df = pd.concat([prediction_df, pd.DataFrame(y_pred, columns = train_y.columns, index = predict_x.index)])
    
    save_predictions(prediction_df, 'XGB', '_Final', working_path)
    print("Run is now fully complete.")
    return prediction_df

def NN_generator(best_params, layers, train_x, train_y):
    
    #Extract the best parameters
    act = best_params['activation']
    units = best_params['units']
    lr = best_params['learning_rate']
    reg = best_params['reg']
    droprate = best_params['dropout']
    epochs = best_params['epochs']
    batch_n = best_params['batch_size']
    es = EarlyStopping(monitor='val_loss', mode='min', patience = 10, restore_best_weights=True)
      
    #Setup the combined df
    input_dim = train_x.shape[1]

    model = Sequential()
    model.add(Dense(units,  kernel_initializer='he_uniform', input_dim = input_dim, activation = act, 
                    kernel_regularizer= regularizers.L1L2(l1=reg, l2=reg)))
    model.add(Dropout(droprate))
    
    for i in range(1, layers):
        model.add(BatchNormalization())
        model.add(Dense(units/(i*2), activation=act,  kernel_initializer='he_uniform',
                        kernel_regularizer= regularizers.L1L2(l1=reg, l2=reg)))
        model.add(Dropout(droprate))
    
    model.add(BatchNormalization())
    model.add(Dense(train_y.shape[1], kernel_initializer='normal',activation='linear'))

    model.compile(optimizer = Adam(learning_rate = lr), loss="mean_squared_error", metrics=["mean_squared_error"])

    history = model.fit(train_x, train_y, validation_split = 0.2, shuffle = True, 
              batch_size = batch_n, epochs = epochs, verbose = 0, callbacks=[es])
   
    return model, history

def NN_evaluator(best_params, layers, train_x, train_y, test_x, test_y):
    model, history = NN_generator(best_params, layers, train_x, train_y)
    iter_pred = model.predict(test_x, verbose = 0)
    return iter_pred, model, history

def backtest_NN(combined_df, min_train_window, re_train_periods, runs):
    dates = sorted(set(list(combined_df.index)))
    prediction_df = pd.DataFrame(columns = [col for col in combined_df.columns if 'Y_' in col])
       
    for i in range(min_train_window,len(dates)):
        train_x, train_y, predict_x, predict_y = generate_df_splits(combined_df, dates[i])
                
        if i % re_train_periods == 0:
            #Train the NN Model
            start_time = time.time()
            study = optimization_instance(objective_NN3, runs)
            train_x, predict_x = apply_robust_scaler(train_x, predict_x)
            y_pred, predict_model, history = NN_evaluator(study.best_params, 3, 
                                                          train_x, train_y, predict_x, predict_y)
            prediction_df = pd.concat([prediction_df, pd.DataFrame(y_pred, columns = train_y.columns, index = predict_x.index)])
            working_path = "C:/Users/andrew_lazzeri/Desktop/MultiAsset Covariance Matrix/"
            save_predictions(prediction_df, 'NN3', str(dates[i])[0:10], working_path)
            save_NN_model(predict_model, 'NN3', str(dates[i])[0:10], working_path)
            end_time = time.time()
            print("One training instance takes: ", (end_time - start_time)/60.0)
            print(str(np.round(i/len(dates) ,2)), " of the run is complete.")
                      
        else:
            train_x, predict_x = apply_robust_scaler(train_x, predict_x)
            y_pred = predict_model.predict(predict_x, verbose = 0)
            prediction_df = pd.concat([prediction_df, pd.DataFrame(y_pred, columns = train_y.columns, index = predict_x.index)])
    
    save_predictions(prediction_df, 'NN3', '_Final', working_path)
    print("Run is now fully complete.")
    return prediction_df

In [6]:
#Run the XGBoost predictions
XG_prediction_df = backtest_XGB(combined_df, initial_train_window, retrain_intervals, CV_budget)
    

One training instance takes:  137.49629204273225
0.09  of the run is complete.
One training instance takes:  141.75285143852233
0.13  of the run is complete.
One training instance takes:  147.19586871465046
0.17  of the run is complete.
One training instance takes:  151.48110322952272
0.21  of the run is complete.
One training instance takes:  155.71312948465348
0.26  of the run is complete.
One training instance takes:  160.53048454920452
0.3  of the run is complete.
One training instance takes:  164.47204701105753
0.34  of the run is complete.
One training instance takes:  170.63983585834504
0.39  of the run is complete.
One training instance takes:  171.92296189864476
0.43  of the run is complete.
One training instance takes:  177.0112081448237
0.47  of the run is complete.
One training instance takes:  180.44828838507334
0.52  of the run is complete.
One training instance takes:  183.93473306894302
0.56  of the run is complete.
One training instance takes:  187.81416420936586
0.6  

In [7]:
#Run the NN predictions
NN3_prediction_df = backtest_NN(combined_df, initial_train_window, retrain_intervals, CV_budget)

9/9 [==============================] - 0s 6ms/step
One training instance takes:  2.6357768177986145
0.09  of the run is complete.
9/9 [==============================] - 0s 4ms/step
One training instance takes:  2.621719717979431
0.13  of the run is complete.
9/9 [==============================] - 0s 6ms/step
One training instance takes:  2.6517183621724447
0.17  of the run is complete.
9/9 [==============================] - 0s 6ms/step
One training instance takes:  2.7085009376207987
0.21  of the run is complete.
9/9 [==============================] - 0s 4ms/step
One training instance takes:  2.7117697397867837
0.26  of the run is complete.
9/9 [==============================] - 0s 6ms/step
One training instance takes:  2.706006371974945
0.3  of the run is complete.
9/9 [==============================] - 0s 6ms/step
One training instance takes:  2.7718979001045225
0.34  of the run is complete.
9/9 [==============================] - 0s 6ms/step
One training instance takes:  2.7781540234

In [8]:
y_df = y_df.add_prefix('Y_')
RNN_synched = X_df.join(y_df)
RNN_synched = RNN_synched.dropna()
X_df = RNN_synched[[col for col in RNN_synched.columns if 'Y_' not in col]]
y_df = RNN_synched[[col for col in RNN_synched.columns if 'Y_' in col]]

In [9]:
def RNN_feeder_array(X_df, y_df, window_size):
    dates = sorted(list(set(X_df.index.get_level_values('Date'))))
    date_refs = []
    RNN_x = []
    RNN_y = []
    for date in dates[window_size::]:
        iter_x = X_df[X_df.index.get_level_values('Date') <= date]
        RNN_y.append(y_df[y_df.index.get_level_values('Date') == date].values)
        RNN_x.append(iter_x.iloc[(-1*window_size*X_df.shape[1])::,:].values)
    return np.array(RNN_y), np.array(RNN_x), dates[window_size::]

train_y, train_x, dates = RNN_feeder_array(X_df, y_df, cov_window)


def array_robust_scaler(train_x, predict_x):
    #Reshape the training array for robust scaler by stacking the array similar to a dataframe
    scaler_train_x = train_x.flatten()
    scaler_train_x = scaler_train_x.reshape(train_x.shape[0]*train_x.shape[1], train_x.shape[2])
    transformer = RobustScaler(quantile_range=(1.0, 99.0)).fit(scaler_train_x)
    scaler_train_x = transformer.transform(scaler_train_x)
    scaler_train_x = scaler_train_x.reshape(train_x.shape[0], train_x.shape[1], train_x.shape[2])
    #Reshape the prediction array like the above
    scaler_predict_x = predict_x.flatten()
    scaler_predict_x = scaler_predict_x.reshape(predict_x.shape[0]*predict_x.shape[1], predict_x.shape[2])
    scaler_predict_x = transformer.transform(scaler_predict_x)
    scaler_predict_x = scaler_predict_x.reshape(predict_x.shape[0], predict_x.shape[1], predict_x.shape[2])
    return scaler_train_x, scaler_predict_x

def array_MSE(y_array, pred_array):
    iter_y = np.nan_to_num(y_array, copy=True, nan=0, posinf=0, neginf=None)
    iter_pred = np.nan_to_num(pred_array, copy=True, nan=0, posinf=0, neginf=None)
    MSE = np.mean(np.power((iter_y - iter_pred), 2))
    return MSE

def ts_RNN_cross_validation(X_array, y_array, model, ts_folds = 4):
    array_len = X_array.shape[0]
    rough_splits = array_len // (ts_folds + 1)
    
    errors = []
    idx = [i for i in range(0, array_len, rough_splits)]

    for i in range(2, len(idx)):
        iter_train_x = X_array[idx[i-2]:idx[i-1]]
        iter_test_x = X_array[idx[i-1]:idx[i]]
        iter_train_y = y_array[idx[i-2]:idx[i-1]]
        iter_test_y = y_array[idx[i-1]:idx[i]]
        iter_train_x, iter_test_x = array_robust_scaler(iter_train_x, iter_test_x)
        
        model.fit(iter_train_x, iter_train_y, verbose=0)
        MSE = array_MSE(iter_test_y, model.predict(iter_test_x, verbose=0))
        errors.append(MSE)
        
    return np.mean(errors)


def objective_CNN(trial, train_x = train_x, train_y = train_y):
    K.clear_session() 
            
    #Setup the model hyperparameter search
    activation_fx = trial.suggest_categorical("activation", ["relu", "selu", "LeakyReLU"])
    conv_filters = trial.suggest_categorical("filters", [3, 5, 7, 9, 11, 13, 15])
    LSTM_layers = trial.suggest_categorical("LSTM_layers", [i for i in range(10, 100, 10)])
    dense_layers = trial.suggest_categorical("Dense_layers", [train_x.shape[2]*i for i in range(10)])
    lr = trial.suggest_categorical('learning_rate', [.001, .01, .1])
    reg = trial.suggest_categorical('reg', [10**-3, 10**-5])
    droprate = trial.suggest_categorical('dropout', [.20, .30, .4])
    epochs = trial.suggest_categorical('epochs', [100])
    batch_n = trial.suggest_categorical('batch_size', [64, 64*2])
    es = EarlyStopping(monitor='val_loss', mode='min', patience = 10, restore_best_weights=True)
    
    #Setup the model structure
    model = Sequential()
    model.add(Conv1D(conv_filters, conv_filters, activation = activation_fx,
                     padding = 'valid', input_shape=(train_x.shape[1], train_x.shape[2])))
    model.add(LSTM(LSTM_layers, activation=activation_fx, return_sequences=True))
    model.add(LSTM(LSTM_layers, activation=activation_fx, return_sequences=True))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(dense_layers, activation=activation_fx, kernel_regularizer= regularizers.L1L2(l1=reg, l2=reg)))
    model.add(Dropout(droprate))
    model.add(BatchNormalization())
    model.add(Dense(int(dense_layers/2), activation=activation_fx, kernel_regularizer= regularizers.L1L2(l1=reg, l2=reg)))
    model.add(Dropout(droprate))
    model.add(BatchNormalization())
    model.add(Dense(train_x.shape[2]*train_x.shape[2], activation='linear'))
    model.add(Reshape((train_x.shape[2], train_x.shape[2])))
    model.compile(optimizer=Adam(learning_rate=lr), loss="mean_squared_error", metrics='mean_squared_error')
    model.fit(train_x, train_y, validation_split = 0.2, shuffle = True, batch_size = batch_n, 
              epochs = epochs, verbose = 0, callbacks=[es]) 
    MSE = ts_RNN_cross_validation(train_x, train_y, model)
    return MSE

def CNN_generator(best_params, train_x, train_y):
    
    #Extract the best parameters
    activation_fx = best_params['activation']
    conv_filters = best_params['filters']
    LSTM_layers = best_params['LSTM_layers']
    dense_layers = best_params['Dense_layers']
    lr = best_params['learning_rate']
    reg = best_params['reg']
    droprate = best_params['dropout']
    epochs = best_params['epochs']
    batch_n = best_params['batch_size']
    es = EarlyStopping(monitor='val_loss', mode='min', patience = 10, restore_best_weights=True)
    
    
    #Setup the model structure
    model = Sequential()
    model.add(Conv1D(conv_filters, conv_filters, activation = activation_fx,
                     padding = 'valid', input_shape=(train_x.shape[1], train_x.shape[2])))
    model.add(LSTM(LSTM_layers, activation=activation_fx, return_sequences=True))
    model.add(LSTM(LSTM_layers, activation=activation_fx, return_sequences=True))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(dense_layers, activation=activation_fx, kernel_regularizer= regularizers.L1L2(l1=reg, l2=reg)))
    model.add(Dropout(droprate))
    model.add(BatchNormalization())
    model.add(Dense(int(dense_layers/2), activation=activation_fx, kernel_regularizer= regularizers.L1L2(l1=reg, l2=reg)))
    model.add(Dropout(droprate))
    model.add(BatchNormalization())
    model.add(Dense(train_x.shape[2]*train_x.shape[2], activation='linear'))
    model.add(Reshape((train_x.shape[2], train_x.shape[2])))
    
    model.compile(optimizer = Adam(learning_rate = lr), loss="mean_squared_error", metrics=["mean_squared_error"])

    history = model.fit(train_x, train_y, validation_split = 0.2, shuffle = True, 
              batch_size = batch_n, epochs = epochs, verbose = 0, callbacks=[es])
   
    return model, history 

def CNN_evaluator(best_params, train_x, train_y, predict_x):
    model, history = CNN_generator(best_params, train_x, train_y)
    iter_pred = model.predict(predict_x, verbose = 0)
    return iter_pred, model, history

"""
def objective_RNN(trial, RNN_train_x=RNN_train_x, RNN_train_y=RNN_train_y):
    K.clear_session() 
            
    #Setup the model hyperparameter search
    activation_fx = trial.suggest_categorical("activation", ["relu", "selu", "LeakyReLU"])
    LSTM_layers = trial.suggest_categorical("LSTM_layers", [i for i in range(10, 100, 10)])
    dense_layers = trial.suggest_categorical("Dense_layers", [train_x.shape[2]*i for i in range(10)])
    lr = trial.suggest_categorical('learning_rate', [.001, .01, .1])
    reg = trial.suggest_categorical('reg', [10**-3, 10**-5])
    droprate = trial.suggest_categorical('dropout', [.20, .30, .4])
    epochs = trial.suggest_categorical('epochs', [100])
    batch_n = trial.suggest_categorical('batch_size', [64, 64*2])
    es = EarlyStopping(monitor='val_loss', mode='min', patience = 10, restore_best_weights=True)
    
    #Setup the model structure
    model = Sequential()
    model.add(LSTM(LSTM_layers, activation=activation_fx, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])))
    model.add(LSTM(LSTM_layers, activation=activation_fx, return_sequences=True))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(dense_layers, activation=activation_fx, kernel_regularizer= regularizers.L1L2(l1=reg, l2=reg)))
    model.add(Dropout(droprate))
    model.add(BatchNormalization())
    model.add(Dense(int(dense_layers/2), activation=activation_fx, kernel_regularizer= regularizers.L1L2(l1=reg, l2=reg)))
    model.add(Dropout(droprate))
    model.add(BatchNormalization())
    model.add(Dense(train_x.shape[2]*train_x.shape[2], activation='linear'))
    model.add(Reshape((train_x.shape[2], train_x.shape[2])))
    model.compile(optimizer=Adam(learning_rate=lr), loss="mean_squared_error", metrics='mean_squared_error')
    model.fit(RNN_train_x, RNN_train_y, validation_split = 0.2, shuffle = True, batch_size = batch_n, 
              epochs = epochs, verbose = 0, callbacks=[es]) 
    MSE = ts_RNN_cross_validation(RNN_train_x, RNN_train_y, model)
    return MSE

def RNN_generator(best_params, train_x, train_y):
    
    #Extract the best parameters
    activation_fx = best_params['activation']
    conv_filters = best_params['filters']
    LSTM_layers = best_params['LSTM_layers']
    dense_layers = best_params['Dense_layers']
    lr = best_params['learning_rate']
    reg = best_params['reg']
    droprate = best_params['dropout']
    epochs = best_params['epochs']
    batch_n = best_params['batch_size']
    es = EarlyStopping(monitor='val_loss', mode='min', patience = 10, restore_best_weights=True)
    
    #Setup the model structure
    model = Sequential()
    model.add(LSTM(LSTM_layers, activation=activation_fx, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])))
    model.add(LSTM(LSTM_layers, activation=activation_fx, return_sequences=True))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(dense_layers, activation=activation_fx, kernel_regularizer= regularizers.L1L2(l1=reg, l2=reg)))
    model.add(Dropout(droprate))
    model.add(BatchNormalization())
    model.add(Dense(int(dense_layers/2), activation=activation_fx, kernel_regularizer= regularizers.L1L2(l1=reg, l2=reg)))
    model.add(Dropout(droprate))
    model.add(BatchNormalization())
    model.add(Dense(train_x.shape[2]*train_x.shape[2], activation='linear'))
    model.add(Reshape((train_x.shape[2], train_x.shape[2])))
    
    model.compile(optimizer = Adam(learning_rate = lr), loss="mean_squared_error", metrics=["mean_squared_error"])

    history = model.fit(train_x, train_y, validation_split = 0.2, shuffle = True, 
              batch_size = batch_n, epochs = epochs, verbose = 0, callbacks=[es])
   
    return model, history 

def RNN_evaluator(best_params, train_x, train_y, predict_x):
    model, history = RNN_generator(best_params, train_x, train_y)
    iter_pred = model.predict(predict_x, verbose = 0)
    return iter_pred, model, history
"""
def backtest_CNN(X_df, y_df, min_train_window, re_train_periods, cov_window, runs):
    #Convert all observations into trailing dates
    y_train, X_train, dates = RNN_feeder_array(X_df, y_df, cov_window)
    prediction_df = pd.DataFrame(columns = [col for col in y_df.columns])
       
    for i in range(min_train_window,len(dates)):
        train_x = X_train[0:(i-1)]
        train_y = y_train[0:(i-1)]
        predict_x = X_train[i]
        predict_y = y_train[i]
        iter_index = y_df[y_df.index.get_level_values('Date') == dates[i]].index
    
       
        """
        if i % re_train_periods == 0:
            #Train the CNN Model
            start_time = time.time()
            study = optimization_instance(objective_CNN, runs)
            train_x, predict_x = array_robust_scaler(train_x, predict_x.reshape(1, predict_x.shape[0], predict_x.shape[1]))
            y_pred, predict_model, history = CNN_evaluator(study.best_params, train_x, train_y, predict_x)
            y_pred = y_pred.reshape(y_df.shape[1], y_df.shape[1])
            prediction_df = pd.concat([prediction_df, pd.DataFrame(y_pred, columns = y_df.columns, index = iter_index)])
            working_path = "C:/Users/andrew_lazzeri/Desktop/MultiAsset Covariance Matrix/"
            save_predictions(prediction_df, 'CNN', str(dates[i])[0:10], working_path)
            save_NN_model(predict_model, 'CNN', str(dates[i])[0:10], working_path)
            end_time = time.time()
            print("One training instance takes: ", (end_time - start_time)/60.0)
            print(str(np.round(i/len(dates) ,2)), " of the run is complete.")
        
        else:
            train_x, predict_x = array_robust_scaler(train_x, predict_x.reshape(1, predict_x.shape[0], predict_x.shape[1]))
            y_pred = predict_model.predict(predict_x, verbose = 0)
            y_pred = y_pred.reshape(y_df.shape[1], y_df.shape[1])
            prediction_df = pd.concat([prediction_df, pd.DataFrame(y_pred, columns = y_df.columns, index = iter_index)])
    
    save_predictions(prediction_df, 'CNN', '_Final', working_path)
    print("Run is now fully complete.")
    """
    return train_x, train_y, predict_x, predict_y

"""
def backtest_RNN(X_df, y_df, min_train_window, re_train_periods, cov_window, runs):
    #Convert all observations into trailing dates
    y_train, X_train, dates = RNN_feeder_array(X_df, y_df, cov_window)
    prediction_df = pd.DataFrame(columns = [col for col in y_df.columns])
    
    for i in range(min_train_window,len(dates)):
        RNN_train_x = X_train[0:(i-1)]
        RNN_train_y = y_train[0:(i-1)]
        RNN_predict_x = X_train[i]
        RNN_predict_y = y_train[i]
        iter_index = y_df[y_df.index.get_level_values('Date') == dates[i]].index
        scaled_train, scaled_predict = array_robust_scaler(RNN_train_x, RNN_predict_x.reshape(1, RNN_predict_x.shape[0], RNN_predict_x.shape[1]))

        if i % re_train_periods == 0:
            #Train the RNN Model
            start_time = time.time()
            study = optimization_instance(objective_RNN, runs)
            y_pred, predict_model, history = RNN_evaluator(study.best_params, scaled_train, RNN_train_y, scaled_predict)
            y_pred = y_pred.reshape(y_df.shape[1], y_df.shape[1])
            prediction_df = pd.concat([prediction_df, pd.DataFrame(y_pred, columns = y_df.columns, index = iter_index)])
            working_path = "C:/Users/andrew_lazzeri/Desktop/MultiAsset Covariance Matrix/"
            save_predictions(prediction_df, 'RNN', str(dates[i])[0:10], working_path)
            save_NN_model(predict_model, 'RNN', str(dates[i])[0:10], working_path)
            end_time = time.time()
            print("One training instance takes: ", (end_time - start_time)/60.0)
            print(str(np.round(i/len(dates) ,2)), " of the run is complete.")
        
        else:
            y_pred = predict_model.predict(scaled_predict, verbose = 0)
            y_pred = y_pred.reshape(y_df.shape[1], y_df.shape[1])
            prediction_df = pd.concat([prediction_df, pd.DataFrame(y_pred, columns = y_df.columns, index = iter_index)])
    
    save_predictions(prediction_df, 'RNN', '_Final', working_path)
    print("Run is now fully complete.")
    return prediction_df

"""
#CNN_prediction_df = backtest_CNN(combined_df[test_list], 252, 252, 1)
#CNN_prediction_df  = backtest_CNN(X_df, y_df, initial_train_window, retrain_intervals, cov_window, 1)

#train_x, train_y, predict_x, predict_y = backtest_CNN(X_df, y_df, initial_train_window, retrain_intervals, cov_window, 1)
#RNN_prediction_df = backtest_RNN(X_df, y_df, initial_train_window, retrain_intervals, cov_window, 1)


'\ndef backtest_RNN(X_df, y_df, min_train_window, re_train_periods, cov_window, runs):\n    #Convert all observations into trailing dates\n    y_train, X_train, dates = RNN_feeder_array(X_df, y_df, cov_window)\n    prediction_df = pd.DataFrame(columns = [col for col in y_df.columns])\n    \n    for i in range(min_train_window,len(dates)):\n        RNN_train_x = X_train[0:(i-1)]\n        RNN_train_y = y_train[0:(i-1)]\n        RNN_predict_x = X_train[i]\n        RNN_predict_y = y_train[i]\n        iter_index = y_df[y_df.index.get_level_values(\'Date\') == dates[i]].index\n        scaled_train, scaled_predict = array_robust_scaler(RNN_train_x, RNN_predict_x.reshape(1, RNN_predict_x.shape[0], RNN_predict_x.shape[1]))\n\n        if i % re_train_periods == 0:\n            #Train the RNN Model\n            start_time = time.time()\n            study = optimization_instance(objective_RNN, runs)\n            y_pred, predict_model, history = RNN_evaluator(study.best_params, scaled_train, RNN_tr

In [10]:
def backtest_CNN(X_df, y_df, min_train_window, re_train_periods, cov_window, runs):
    #Convert all observations into trailing dates
    y_train, X_train, dates = RNN_feeder_array(X_df, y_df, cov_window)
    prediction_df = pd.DataFrame(columns = [col for col in y_df.columns])
    
    
    for i in range(min_train_window,len(dates)):
        train_x = X_train[0:(i-cov_window)] #Avoid overlaps with any of the predict window
        train_y = y_train[0:(i-cov_window)] #Avoid overalps with any of the predict window
        predict_x = X_train[i]
        predict_x = predict_x.reshape(1, predict_x.shape[0], predict_x.shape[1])
        predict_y = y_train[i]
        iter_index = y_df[y_df.index.get_level_values('Date') == dates[i]].index
        
        if i % re_train_periods == 0:
            #Train the CNN Model
            start_time = time.time()
            study = optimization_instance(objective_CNN, runs)
            #scaled_train, scaled_predict = array_robust_scaler(train_x, predict_x.reshape(1, predict_x.shape[0], predict_x.shape[1]))
            y_pred, predict_model, history = CNN_evaluator(study.best_params, train_x, train_y, predict_x)
            y_pred = y_pred.reshape(y_df.shape[1], y_df.shape[1])
            prediction_df = pd.concat([prediction_df, pd.DataFrame(y_pred, columns = y_df.columns, index = iter_index)])
            working_path = "C:/Users/andrew_lazzeri/Desktop/MultiAsset Covariance Matrix/"
            save_predictions(prediction_df, 'CNN', str(dates[i])[0:10], working_path)
            save_NN_model(predict_model, 'CNN', str(dates[i])[0:10], working_path)
            end_time = time.time()
            print("One training instance takes: ", (end_time - start_time)/60.0)
            print(str(np.round(i/len(dates) ,2)), " of the run is complete.")
        
        else:
            #scaled_train, scaled_predict = array_robust_scaler(train_x, predict_x.reshape(1, predict_x.shape[0], predict_x.shape[1]))
            y_pred = predict_model.predict(predict_x, verbose = 0)
            y_pred = y_pred.reshape(y_df.shape[1], y_df.shape[1])
            prediction_df = pd.concat([prediction_df, pd.DataFrame(y_pred, columns = y_df.columns, index = iter_index)])
        
    save_predictions(prediction_df, 'CNN', '_Final', working_path)
    print("Run is now fully complete.")
    
    return y_pred, predict_model, history, train_x, predict_x, study


In [ ]:
y_pred, predict_model, history, train_x, predict_x, study = backtest_CNN(X_df, y_df, initial_train_window, retrain_intervals, cov_window, CV_budget)

One training instance takes:  889.7617967327436
0.08  of the run is complete.


One training instance takes:  1799.7092478871346
0.12  of the run is complete.


In [ ]:
pd.DataFrame(y_pred)

In [ ]:
test = pd.read_csv("C:/Users/andrew_lazzeri/Desktop/MultiAsset Covariance Matrix/Prediction_dfs/CNN__Final.csv")

In [ ]:
test[['Date', 'Commodity']] = test['Unnamed: 0'].str.split(',', expand =True)

In [ ]:
test = test.set_index(['Date', 'Commodity'])

In [ ]:
iter_cols = list(set(test.index.get_level_values('Commodity')))

for col in iter_cols:
    iter_df = test[test.index.get_level_values('Commodity') == col]
    for col2 in iter_df.columns:
        plt.plot(iter_df[col2].values)
        plt.title(col2)
        plt.show()
    